# Supervised Learning - Classification Problem

## Students' dropout and academic success

### (Droupout, Enrolled, Graduate)

Faculdade: FEUP - Faculdade de Engenharia da Universidade do Porto

Curso: L.EIC - Licenciatura em Engenharia Informática e Computação 

Unidade Curricular: Inteligência Artificial

Ano Curricular: 2021/22

Grupo: 21_1D

Elementos:
- Henrique Ribeiro Nunes, up201906852@up.pt
- Margarida Assis Ferreira, up201905046@up.pt
- Patrícia do Carmo Nunes Oliveira, up201905427@up.pt

### Especificação

O foco principal deste problema é analisar a informação conhecida sobre a matrícula do aluno (percurso acadêmico, demografia e fatores socioeconómicos) e o desempenho académico dos alunos no final do primeiro e segundo semestres. Com o objetivo de usar estes dados para construir modelos de classificação para **prever a desistência e o sucesso académico dos alunos**.

Este problema é um ***single label multiclass classification problem*** com 37 atributos:
- 36 métricas distintas para descrever as informações do aluno.
- 1 objetivo com 3 resultados possíveis (*Droupout*, *Enrolled*, *Graduate*).

Existe um **forte desbalanceamento** em relação a um dos resultados possíveis.


### Tools & Resources

// TODO AQUI 

## Data Analysis

A análise dos dados é um passo importante nos problemas de classificação. 

Nesta secção são analisados os atributos que classificam os dados e o tipo e intervalo de valores de cada um dos atributos, bem como o tamanho do conjunto de dados, a presença de valores nulos ou amostras duplicadas. Igualmente é explorado a distribuição das classes e de valores por atributos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
SAVE_FILE = False

In [ ]:
# Load data
data = pd.read_csv("data/data_original.csv")
pd.set_option('max_columns', None)

In [ ]:
# Print head values and summary statistics
print(data.describe())
print()
print(data.head())
print()

In [ ]:
# Get all 36 features used and 3 possible results
features = list(data.columns)
features.remove("class")
print("Number of Features: {}".format(len(features)))
print("Features: {}".format(features))
values = list(data[features].values)

classes = list(data["class"].unique())
classes.sort()
print("\nNumber of Results: {}".format(len(classes)))
print("Result/Prediction: {}".format(classes))
targets = list(data["class"].values)

In [ ]:
# Check atribute types and values interval
for name, dtype in data.dtypes.iteritems():
    print("{} | {} | [{} , {}] ".format(name.ljust(46), str(dtype).ljust(7), data[name].min(), data[name].max()))

In [ ]:
# Data size
print("Data Size: {}".format(len(data)))

In [ ]:
# Check if there are columns with N/A values
print("N/A values found: {}".format(data.isnull().values.any()))

In [ ]:
# Check if there are duplicated Data
bool_series = data.duplicated()
print(bool_series)

old_size = len(data)

# Removing all duplicated data if exists
data = data[~bool_series] 

new_size = len(data)

# check if there were actualy duplicated data
print()
if (new_size == old_size):
    print("No data was removed: there were no duplicated data")
else:
    print("Was found and removed {} duplicated data".format(old_size-new_size))

In [ ]:
# Resultant data without duplicated entries
data

In [ ]:
# Count each class
def countEachClass(data):
    n_dropouts = data['class'].value_counts().Dropout
    n_enrolled = data['class'].value_counts().Enrolled
    n_graduate = data['class'].value_counts().Graduate
    return n_dropouts, n_enrolled, n_graduate

In [ ]:
def sampleEachClass(data):
    class_dropout = data[data['class'] == "Dropout"]
    class_enrolled = data[data['class'] == "Enrolled"]
    class_graduate = data[data['class'] == "Graduate"]
    return class_dropout, class_enrolled, class_graduate

In [ ]:
# Class Distribution
# Check data balance
n_dropouts, n_enrolled,n_graduate = countEachClass(data)
print("Number of 'Dropout' occurences: {}".format(n_dropouts))
print("Number of 'Enrolled' occurences: {}".format(n_enrolled))
print("Number of 'Graduate' occurences: {}".format(n_graduate))
print()

# Corresponding plot 
count_result = pd.DataFrame(data["class"]).value_counts().rename_axis("class").reset_index(name="count")
print(count_result)

# Bar plot
plot_count_res = sb.barplot(data=count_result, x="count", y="class")
plt.show()

# Pie plot with percentages
plt.pie([n_graduate, n_dropouts, n_enrolled], autopct = '%0.00f%%')
plt.show()

In [ ]:
# Histogram for each feture values and respective count of classses

def count_targets_for_each_value(data, feat):
    data_aux = data[[feat, "class"]]
    d = {}
    for line in data_aux.values:
        if not line[0] in d:
            d[line[0]] = {classes[0]: 0, classes[1]: 0, classes[2]: 0}
        d[line[0]][line[1]] += 1

    for entry in d.keys():
        d[entry] = [d[entry][c] for c in classes]
    aux = pd.DataFrame(d, index=classes)
    return aux.reindex(sorted(aux.columns), axis=1)


for i in range(36):
    plt.title(features[i])
    plt.hist(data[features[i]].values)
    plt.show()
    print(count_targets_for_each_value(data, features[i]))

Analisando a distribuição de cada atributo pelas classes possíveis, podemos concluir que existem alguns atributos irrelevantes, na medida que não se distingue à priori nem se obtem qualquer informação sobre qual a classe mais provável para uma nova amostra com base no valor desse atributo, como conseguimos identifificar no caso do atributo 'Curricular units 2nd sem (credited)'

In [ ]:
# Distribution of each attribute 
def densityPlot(x, hue, fill):
    sb.displot(data, x=x, hue=hue, kind="kde", fill=fill)
    
# all 36 distinct attributes
attributes = list(data.columns)
attributes.remove("class")
    
for attribute in attributes:
    densityPlot(attribute, 'class', True)

In [ ]:
# Correlation Heatmap

corr = data.corr()

sb.set(rc = {'figure.figsize': (15,15) })
ax = sb.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sb.diverging_palette(20, 240, n=200),
    square=True,
    xticklabels=True,
    yticklabels=True,
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);
plt.show()

sb.reset_orig()

**Propriedades do Problema:** (a partir da análise dos dados de entrada)

- Nominal and Discrete attributes (including some binary ones)
- Dimensionality = 37 attibutos
- Size = 4424
- Type = Data Matrix
- No missing or duplicate Data
- No meaningful outliers
- Imbalance data

## Pré-processamento dos dados

Tendo em conta as conclusões obtidas pela a análise dos dados originais mostrada anteriormente, podemos apercebermo-nos que é necessário realizar um pré-processamento dos dados, com o objetivo de resolver o maior problema adjacente a estes: a falta de balanceamento entre as diferentes classes de alvo. 

Para tal podemos usar entre outras estratégias uma das seguintes:
- **oversampling** : «Aumentar o número de amostras/entradas da menor classe até coincidir com o tamanho da maior classe»
- **undersampling** : «Diminuir o número de amostras/entradas da maior classe até coincidir com o tamanho da menor classe»

Em qualquer uma das abordagens acima, a escolha das amostras escolhidas para serem retidas ou replicadas com pequenas modificações é aleatória.

### Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

def standardize(data, to_standardize):
    data_to_standardize = data[to_standardize]
    scaler = StandardScaler()
    stand_values = scaler.fit_transform(data_to_standardize.values)

    stand_values_df = pd.DataFrame(
        stand_values, 
        index=data_to_standardize.index, 
        columns=to_standardize)
    data[to_standardize] = stand_values_df[to_standardize]
    return data

In [ ]:
non_binary_features = [feature for feature in features if len(data[feature].unique()) != 2]
data_standard = standardize(data.copy(), non_binary_features)

### Undersampling

Uma das técnicas para lidar com o desbalanceamento de classes em machine lerning é chamado de *undersampling*. Esta técnica de balanceamento consiste em remover algumas observações das classes majoritárias, até que as classes majoritárias e minoritárias sejam equilibradas. A técnica *undersampling* pode ser uma boa escolha quando temos dados desequilibrados, mas uma desvantagem é que removemos informações que podem ser valiosas.

Para remover as observações das classes majoritárias, usamos a função `sample(sequence, k)`, uma função do módulo `Random` de Python, que retorna uma lista de comprimento `k` de itens escolhidos aleatoriamente de `sequence`.

In [ ]:
# Imbalance Original Data
print("Classes count:")
print(data['class'].value_counts())

data['class'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
def undersampling(data):
    n_dropouts,n_enrolled,n_graduate = countEachClass(data)
    class_dropout,class_enrolled,class_graduate = sampleEachClass(data)
    
    class_dropout_under = class_dropout.sample(n_enrolled, replace=True)
    class_graduate_under = class_graduate.sample(n_enrolled, replace=True)
    return pd.concat([class_dropout_under, class_graduate_under, class_enrolled], axis=0)

Agora temos os nossos dados balanceados, como é possível observer no gráfico criado pelo código abaixo.

In [ ]:
data_under = undersampling(data)

# plot the count after under-sampeling
print("Classes count after under-sampling:")
print(data_under['class'].value_counts())

data_under['class'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
# Save to file
if SAVE_FILE: data_under.to_csv("data/data_under.csv", index=False)

### Oversampling

As entradas da classe menor sáo replicadas até totalizarem o número de amostras da classe maior.

In [ ]:
n_dropouts,n_enrolled,n_graduate = countEachClass(data)

print("DROPOUT: {} | ENROLLED: {} | GRADUATE: {}".format(n_dropouts, n_enrolled, n_graduate))

In [ ]:
# Imbalance Original Data
unbalanced_count = data['class'].value_counts()
unbalanced_count.plot.bar()
plt.show()

#### Random Over-Sampling

«Oversampling can be defined as adding more copies to the minority class.»

**Desvantagens:** pode causar *overfitting* e pobre generalização do conjunto de dados para teste.

In [ ]:
def oversampling(data):
    n_dropouts,n_enrolled,n_graduate = countEachClass(data)
    dropout_samples,enrolled_samples,graduate_samples = sampleEachClass(data)

    dropout_samples_over = dropout_samples.sample(n_graduate, replace=True)
    enrolled_samples_over = enrolled_samples.sample(n_graduate, replace=True)

    return pd.concat([graduate_samples, dropout_samples_over, enrolled_samples_over], axis=0)

In [ ]:
data_over = oversampling(data)
print("Total dintinct classes: \n{}".format(data_over['class'].value_counts()))

rnd_oversampling_count = data_over['class'].value_counts()
rnd_oversampling_count.plot.bar()
plt.show()

In [ ]:
# Save to file
if SAVE_FILE: data_over.to_csv("data/data_over.csv", index=False)

É importante referir que nenhuma das soluções acima é uma solução perfeita, pois a aplicação de undersampling pode implicar a perda de infromação, da mesma forma que a aplicação de oversampling (sem qualquer modificação das amostras escolhidas aleatóriamente para serem replicadas) pode levar a um posterior overfitting dos modelos gerados a estes novos dados.

### Combine under and over sampling

Tendo isto em conta a seguinte tentativa tenta encontrar um meio termos entre as soluções anteriores, fazendo as classes em questão convergir para um valor mediano e não para um máximo nem minímo, tentando combater as consequencias sentidas nos dados ao aplicar isoladamente cada uma das estratégias, obtando por alcaçar um meio termo.

In [ ]:
# make the counts meet at the middle point
# in this case the middle point is consider to be the dropout class
def combine_sampling(data):
    n_dropouts,n_enrolled,n_graduate = countEachClass(data)
    dropout_samples,enrolled_samples,graduate_samples = sampleEachClass(data)
    graduate_samples_combine = graduate_samples.sample(n_dropouts, replace=True)
    enrolled_samples_combine = enrolled_samples.sample(n_dropouts, replace=True)
    return pd.concat([graduate_samples_combine, dropout_samples, enrolled_samples_combine], axis=0)

In [ ]:
data_combine = combine_sampling(data)

print("Total dintinct classes: \n{}".format(data_combine['class'].value_counts()))

combine_count = data_combine['class'].value_counts()
combine_count.plot.bar()
plt.show()

In [ ]:
# Save to file
if SAVE_FILE: data_combine.to_csv("data/data_combine.csv", i)

## Learning Algorithms

### Identification of the Target Concept

A pergunta a que queremos responder é a seguinte: "Tendo em conta o percurso académico de um aluno e outros fatores externos como é que conseguimos prever se este vai desistir, graduar ou continuar no mesmo ano?"

De forma a avaliar os modelos posterioemente criados podemos escolher nesta secção se os dados a usar ai foram proviamente standarizados e qual o tipo de estratégia para corrigir o balanceamento que é utilizada (inclusive nenhuma). 

In [ ]:
data = pd.read_csv('data/data_original.csv')

non_binary_features = [feature for feature in features if len(data[feature].unique()) != 2]

data = standardize(data, non_binary_features)

data = undersampling(data)
# data = oversampling(data)
# data = combine_sampling(data)

values = list(data[list(data.columns[:-1])].values)
targets = list(data['class'].values)

GRIDSEARCH = False

### Validation

#### Train test split

In [ ]:
from sklearn.model_selection import train_test_split
def split_data(data, classes, test_size):
  feat_train, feat_test, target_train, target_test = train_test_split(data, classes, test_size=test_size, shuffle=True)
  return feat_train, feat_test, target_train, target_test

#### Cross validation

In [ ]:
from sklearn.model_selection import cross_val_score
def cross_validation(model, features, targets, cv):
    scores = cross_val_score(model, features, targets, cv=cv)
    return scores

#### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def confusion_matrix(target_test, target_predictions):    
    return metrics.confusion_matrix(target_test, target_predictions)

def display_confusion_matrix(cm):
    metrics.ConfusionMatrixDisplay(cm, display_labels=classes).plot()

#### Validate model

In [87]:
from sklearn import metrics

def precision(cm, i):
    return cm[i][i]/sum(cm[i])

def recall(cm, i):
    cmt = np.copy(cm).transpose()
    return cmt[i][i]/sum(cmt[i])

def f_measure(cm, i):
    p = precision(cm, i)*100
    r = recall(cm, i)*100
    return 2 * (p * r) / (p + r)

def validate(model, features, targets):
    test_size = 0.2
    cross_validation_split = 5
    feat_train, feat_test, target_train, target_test = split_data(features, targets, test_size)
    
    model.fit(feat_train, target_train)
    predictions = model.predict(feat_test)
    
    acc = metrics.accuracy_score(target_test, predictions)
    print("Accuracy: {:.2f}\n".format(acc*100))
    
    cm = confusion_matrix(target_test, predictions)
    print("Precision:")
    for i in range(3): print("\t{:.2f}% - {}".format(precision(cm, i)*100, classes[i]))
    print("Recall:")
    for i in range(3): print("\t{:.2f}% - {}".format(recall(cm, i)*100, classes[i]))
    print("F1-measure:")
    for i in range(3): print("\t{:.2f}% - {}".format(f_measure(cm, i), classes[i]))
    
    display_confusion_matrix(cm)
    
    scores = cross_validation(model, features, targets, cross_validation_split)
    print("Cross validation {}-fold: {}".format(cross_validation_split, [round(x, 3) for x in scores]))
    print("\t{:.2f} accuracy with a standard deviation of {:.2f}".format(scores.mean(), scores.std()))

In [ ]:
from sklearn.model_selection import GridSearchCV

def grid_search(model, grid_params, features, targets):
    test_size = 0.2
    cross_validation_split = 5
    feat_train, feat_test, target_train, target_test = split_data(features, targets, test_size)
    
    gs = GridSearchCV(
        model,
        grid_params,
        verbose = 1,
        n_jobs = 1,
        cv = cross_validation_split,
    )
    
    gs_results = gs.fit(feat_train, target_train)
    
    print("best score: " + str(gs_results.best_score_))
    print("best estimator: " + str(gs_results.best_estimator_))
    print("best parameters: " + str(gs_results.best_params_))
    
    return gs_results.best_params_

### Algorithms

#### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

grid_params_dtc = {
    'criterion': ['gini', 'entropy', 'log_loss'], 
    'splitter': ['best', 'random'],
    'max_depth': [5, 6, 7, 8, 9, 10, 15, 20, 50],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 5, 10, 20],
}

def_params_dtc = {'criterion': 'entropy', 'max_depth': 7, 'max_features': None, 'max_leaf_nodes': None, 'splitter': 'random'}
best_params_dtc = grid_search(DecisionTreeClassifier(), grid_params_dtc, values, targets) if GRIDSEARCH else def_params_dtc

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create the classifier
dtc = DecisionTreeClassifier(
  criterion= best_params_dtc['criterion'], 
  splitter= best_params_dtc['splitter'], 
  max_depth= best_params_dtc['max_depth'], 
  max_features = best_params_dtc['max_features'], 
  max_leaf_nodes= best_params_dtc['max_leaf_nodes']) 
validate(dtc, values, targets)

In [ ]:
# Plot tree
from sklearn.tree import plot_tree
plot_tree(dtc)

In [ ]:
# Create PDF with the tree
import graphviz
from sklearn.tree import export_graphviz
'''
dot_data = export_graphviz(dtc, out_file=None, feature_names=features, class_names=targets ) 
graph = graphviz.Source(dot_data) 
graph.render("decision_tree") 
'''

#### K-nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

grid_params_knn = {
    'n_neighbors': [3,5,11,19], 
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
}

def_params_knn = {'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
best_params_knn = grid_search(KNeighborsClassifier(), grid_params_knn, values, targets) if GRIDSEARCH else def_params_knn 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier(best_params_knn['n_neighbors'],
                           weights = best_params_knn['weights'],
                           metric = best_params_knn['metric'])
validate(knc, values, targets)

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.inspection import DecisionBoundaryDisplay
'''
_, ax = plt.subplots()
DecisionBoundaryDisplay.from_estimator(
    clf,
    X=np.array([line[:2] for line in data_train]),
    ax=ax,
    response_method="predict",
    plot_method="pcolormesh",
    shading="auto",
)

sns.scatterplot(
        x=data[labels[0]],
        y=data[labels[1]],
        alpha=1.0,
        edgecolor="black",
    )
plt.show()
'''

#### Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

grid_params_mlp = {
    'solver': ['lbfgs', 'sgd', 'adam'], 
    'alpha': [1e-5, 1e-1],
    'hidden_layer_sizes': [(5, 2)],
    'max_iter':[5000, 2500],
    'random_state':[None, 1],
}

def_params_mlp = {'alpha': 0.1, 'hidden_layer_sizes': (5, 2), 'max_iter': 5000, 'random_state': 1, 'solver': 'sgd'}
best_params_mlp = grid_search(MLPClassifier(), grid_params_mlp, values, targets) if GRIDSEARCH else def_params_mlp

In [ ]:
from sklearn.neural_network import MLPClassifier

nnc = MLPClassifier(
    solver = best_params_mlp['solver'], 
    alpha = best_params_mlp['alpha'],
    hidden_layer_sizes = best_params_mlp['hidden_layer_sizes'],
    max_iter = best_params_mlp['max_iter'],
    random_state = best_params_mlp['random_state'])

validate(nnc, values, targets)

### Evaluation of the learning process

No que diz respeito à avaliação do processo de aprendizagem foi implementada a função `validate` (apresentada na secção de *Validation*) que é responsável por dividir os dados em dados de treino e de teste, bem como avaliar os resultados obtidos em cada um dos modelos criados usando as métricas apropriadas: ***accuracy***, ***precision***, ***recall***, ***F-measure*** e ***confusion matrix***, tendo sempre em consideração as características do problema: o problema trata-se um um problema de classificação ***multiclass single label*** com 3 diferentes classes objetivo (3 *targets*). Esta avaliação centra-se principalmente no conjunto de dados de teste.

### Results Comparision

O resultados apresentados foram obtidos apartir da comparaação da avaliação de performance dos diferentes modelos considerando as seguintes combinações nas variações da data de entrada:

- aplicação de **standarização** aos dados: *yes/no*
- estratégia para **tratamento do desbalanceamento** dos dados: *none/under/over/combined*
- **gridsearch** para escolha dos melhores parametros a usar para cada algoritmo: *yes/no*

NOTA: Estes parametros podem ser ajustados e configurados no início da secção ***Learning Algorithms, «Identification of the Target Concept»***.

Neste sentido foram comparados os resultados obtidos para cada uma das combinações possíveis para os 3 algoritmos apresentados anteriormente : ***Decision Tree Classifier***, ***K-Nearest Neighbor***, ***Neural Networks***, resultando nos seguintes gráficos e tabelas apresentados em seguida.